In [1]:
# importing libraries 
import pandas as pd
import torch
import random
from tqdm import tqdm
from torch.utils.data import DataLoader
import itertools
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import os
from datasets import Dataset
# import packages (remember to import the local_transoformers package in your folder correctly!)
from local_transformers_clinical_BERT.src.transformers import (
    TrainingArguments,
    Trainer
) 
import pandas as pd
import torch
import random
from datasets import Dataset
from torch.utils.data import DataLoader
import itertools
from torch.utils.data import TensorDataset
import tensorflow as tf
from sklearn.metrics import make_scorer, average_precision_score
from local_transformers_clinical_BERT.src.transformers.models.CustomClinicalBERT import CustomClinicalBertForCombinedLearning # reads the local_transformers folder instead of the actual transformers package
from local_transformers_clinical_BERT.src.transformers.CustomComputeLoss import CustomTrainer # reads the local_transformers folder instead of the actual transformers package
from transformers import AutoModel
import os
import pickle
from local_transformers_clinical_BERT.src.transformers.trainer_callback import TrainerCallback
import matplotlib.pyplot as plt
import math
import numpy as np
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import RepeatedKFold, cross_val_predict, cross_val_score
import numpy as np
from numpy import mean, std
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
from tqdm import tqdm
import statistics
import numpy
from local_transformers_clinical_BERT.src.transformers.data.data_collator import DataCollatorForLanguageModeling
from local_transformers_clinical_BERT.src.transformers.models.auto.tokenization_auto import AutoTokenizer
from sklearn.model_selection import KFold

if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is not available, running on CPU')

df=pd.read_csv("df.csv")
df["death_in_30"] = df["death_in_30"].astype(int)
df["DVT"] = df["DVT"].astype(int)
df["PE"] = df["PE"].astype(int)
df["post_aki_status"] = df["post_aki_status"].astype(int)
df["PNA"] = df["PNA"].astype(int)
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = CustomClinicalBertForCombinedLearning.from_pretrained("medicalai/ClinicalBERT",output_hidden_states=True,weights=None,lambda_constant=5) #NOTE: Change lambda constant to the appropriate one


# preparing the clinical notes for training and fine-tuning 
# this function here helps set up and prepare the data for fine-tuning, including those of the additional labels. 
def tokenize_and_prepare(batch,text_col,outcome_col):
    tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
    # note max_length was 64. 
    encodings = tokenizer(batch[text_col], padding=True, truncation=True, max_length=296) 
    # print(f'batch[outcome_col]: {batch[outcome_col]}')
    additional_labels = torch.tensor(batch[outcome_col], dtype=torch.float)  # note this line ensures that the additional labels are considered during semi-supervised fine-tuning
    # print(f'additional_labels: {additional_labels}') 
    # print("additional_labels shape:", additional_labels.shape)
    return {"input_ids": encodings["input_ids"], "attention_mask": encodings["attention_mask"], "additional_labels": additional_labels, "labels": encodings["input_ids"]}

def prepare_data(df, text_col, outcome_col):
    texts = list(df[text_col])
    outcomes = list(df[outcome_col])
    data = {text_col: texts, outcome_col: outcomes}
    dataset = Dataset.from_dict(data)
    dataset = dataset.map(lambda batch: tokenize_and_prepare(batch, text_col, outcome_col), batched=True, batch_size=len(texts))
    dataset.set_format("torch", columns=["input_ids", "attention_mask", "additional_labels", "labels"])
    return dataset




2023-10-09 03:29:32.861789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 03:29:33.334284: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-09 03:29:33.334329: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-09 03:29:33.334333: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing CustomClinicalBertForCombinedLearning: ['distilbert.transformer.layer.5.ffn.lin2.weight', 'distilbert.transformer.layer.4.attention.v_lin.weight', 'distilbert.transformer.layer.3.output_layer_norm.bias', 'distilbert.transformer.layer.4.attention.out_lin.weight', 'distilbert.transformer.layer.2.sa_layer_norm.bias', 'distilbert.transformer.layer.2.attention.k_lin.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.transformer.layer.3.attention.k_lin.weight', 'distilbert.transformer.layer.4.attention.k_lin.weight', 'distilbert.embeddings.word_embeddings.weight', 'distilbert.transformer.layer.1.attention.q_lin.weight', 'distilbert.transformer.layer.2.ffn.lin1.weight', 'distilbert.transformer.layer.5.ffn.lin1.weight', 'distilbert.tran

In [2]:
# this fine-tunes the model with the given parameters
def get_model(name, train_dataset,val_dataset,old_weights=None):
    tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
    model = CustomClinicalBertForCombinedLearning.from_pretrained("medicalai/ClinicalBERT",output_hidden_states=True,weights=None,lambda_constant=5) #NOTE: Change lambda constant to the appropriate one

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=7,
        per_device_train_batch_size=36,
        per_device_eval_batch_size=36,
        warmup_steps=1500,
        weight_decay=0.001,
        learning_rate=0.00001,
        logging_dir=None,
        logging_steps=1000,
        save_strategy="epoch")




    # Define the data collator for masked language modeling
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
    # Instantiate the Trainer
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer
        )
    trainer.train()
    # trainer.evaluate(eval_dataset=val_dataset)
    trainer.model.save_pretrained(name) 
    trainer.tokenizer.save_pretrained(name) 

In [3]:
# this gets the embeddings of the specified tuned model
def get_clinicalBERT_updated(texts, name):
    notes = list(texts)
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = CustomClinicalBertForCombinedLearning.from_pretrained(name, output_hidden_states=True)

    # Check if CUDA is available and if so, move the model to GPU
    if torch.cuda.is_available():
        model = model.to("cuda")

    # Tokenize texts and create input IDs and attention masks
    input_ids, attention_masks = [], []
    for text in notes:
        encoded_input = tokenizer.encode_plus(text, max_length=128, padding='max_length', truncation=True, 
                                              return_attention_mask=True, return_tensors='pt')  # change max_length if needed [NOTE: max_length may differ across task]
        input_ids.append(encoded_input['input_ids'])
        attention_masks.append(encoded_input['attention_mask'])

    # Convert lists to tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    # If CUDA is available, move the tensors to GPU
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")
        attention_masks = attention_masks.to("cuda")

    # Batch size for processing
    batch_size = 32
    # Initialize an empty list to store embeddings
    embeddings = []
    # Process input IDs and attention masks in batches
    for i in tqdm(range(0, len(input_ids), batch_size)):
        # print("progress:", (i/len(input_ids))*100)
        input_ids_batch = input_ids[i:i+batch_size]
        attention_masks_batch = attention_masks[i:i+batch_size]

        with torch.no_grad():
            # Get the model's output
            output = model(input_ids=input_ids_batch, attention_mask=attention_masks_batch)

        # Retrieve the last_hidden_state from the output
        last_hidden_state = output.hidden_states[-1]

        # Optionally, you can obtain the pooled_output by averaging or max-pooling the last_hidden_state
        pooled_output = torch.mean(last_hidden_state, dim=1)

        # If CUDA is available, move the embeddings back to CPU
        if torch.cuda.is_available():
            pooled_output = pooled_output.to("cpu")

        # Add the embeddings to the list
        embeddings.extend(pooled_output.numpy())

    embeddings = np.array(embeddings)
    return embeddings


In [4]:
def K_fold_val(outcome_col, df):
    
    skf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []

    # Define the parameter grid
    param_grid = {
        'learning_rate': [0.1,0.15,0.3],
        'max_depth': [4,5,6,7,8],
        'min_child_weight':[1,2,4]
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df)):
        # Split the data into train/test sets
        train, test = df.iloc[train_index], df.iloc[test_index]
        # Create a new column for embeddings based on the training set only
        name=f"{outcome_col}_fold_{i}"
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_notes=(train_new.reset_index(drop=True))
            val_notes=(val_new.reset_index(drop=True))
            train_dataset = prepare_data(train_notes, "AN_PROC_NAME", outcome_col)
            val_dataset = prepare_data(val_notes, "AN_PROC_NAME", outcome_col)   
            get_model(name, train_dataset,val_dataset)
        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_clinicalBERT_updated(train["AN_PROC_NAME"], name)
            X_test = get_clinicalBERT_updated(test["AN_PROC_NAME"], name)
            # X_train=get_clinicalBERT_updated(train["AN_PROC_NAME"], name)
            # X_test = get_clinicalBERT_updated(test["AN_PROC_NAME"], name)
        
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)


        # Prepare training and testing data
        y_train = train[outcome_col]
        y_test = test[outcome_col]

        model = xgb.XGBClassifier(random_state=42,tree_method='gpu_hist')
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]
        #model.fit(X_train,y_train)
        #y_pred = model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")


In [5]:
results=K_fold_val("death_in_30", df) 

0it [00:00, ?it/s]You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing CustomClinicalBertForCombinedLearning: ['distilbert.transformer.layer.2.ffn.lin1.weight', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.3.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.1.output_layer_norm.weight', 'distilbert.transformer.layer.3.ffn.lin1.bias', 'distilbert.transformer.layer.3.ffn.lin2.bias', 'distilbert.transformer.layer.5.attention.q_lin.bias', 'distilbert.transformer.layer.2.attention.q_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'vocab_layer_norm.weight', 'distilbert.transformer.layer.4.attention.q_lin.bias', 'distilbert.transformer.layer.4.ffn.lin2.weight', 'distilbert.transform

test


Step,Training Loss
1000,11.388400
2000,7.420700
3000,6.261900
4000,5.679800
5000,5.319200
6000,5.040900
7000,4.794400
8000,4.670500
9000,4.534700


100%|██████████| 531/531 [01:02<00:00,  8.48it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

test


Step,Training Loss
1000,11.328500
2000,7.464500
3000,6.310000
4000,5.702900
5000,5.360400
6000,5.068900
7000,4.822500
8000,4.678400
9000,4.604200


100%|██████████| 531/531 [01:06<00:00,  8.02it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [8:35:38, 6187.79s/it]


In [6]:
print(results) # lambda=10

    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     num_train_epochs=6,
    #     per_device_train_batch_size=36,
    #     per_device_eval_batch_size=36,
    #     warmup_steps=1500,
    #     weight_decay=0.001,
    #     learning_rate=0.00001,
    #     logging_dir="./logs",
    #     logging_steps=1000,
    #     save_strategy="epoch")

metrics: Mean AUROC: 0.8635091881524616, SD AUROC: 0.006164536565839913, Mean AUPRC: 0.17786813125432843, SD AUPRC: 0.02434116846840453


## Part II: DVT

In [5]:
results=K_fold_val("DVT", df) # lambda = 5

0it [00:00, ?it/s]You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing CustomClinicalBertForCombinedLearning: ['distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.3.attention.q_lin.bias', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.2.sa_layer_norm.bias', 'distilbert.transformer.layer.4.attention.v_lin.weight', 'distilbert.transformer.layer.2.ffn.lin2.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.1.attention.v_lin.bias', 'distilbert.transformer.layer.2.attention.out_lin.weight', 'distilbert.transformer.layer.1.ffn.lin1.bias', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.transformer.layer.5.sa_layer_norm.weight', 'distilb

test


Step,Training Loss
1000,8.690500
2000,5.435600
3000,4.688000
4000,4.213300
5000,3.905200
6000,3.650300
7000,3.454800
8000,3.316500
9000,3.215400
10000,3.150900


100%|██████████| 531/531 [00:53<00:00,  9.91it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

test


Step,Training Loss
1000,8.682000
2000,5.441500
3000,4.670200
4000,4.200500
5000,3.864400
6000,3.650200
7000,3.424700
8000,3.296100
9000,3.179400
10000,3.107400


100%|██████████| 531/531 [02:31<00:00,  3.50it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=  12.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=  12.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=  12.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=  12.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=  11.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   7.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   7.9s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [12:37:45, 9093.14s/it] 


In [6]:
print(results) 

    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     num_train_epochs=7,
    #     per_device_train_batch_size=40,
    #     per_device_eval_batch_size=32,
    #     warmup_steps=1000,
    #     # weight_decay=0.1,
    #     learning_rate=0.00001,
    #     logging_dir="./logs",
    #     logging_steps=1000,
    #     save_strategy="epoch"), max_length=296, lambda = 1 

metrics: Mean AUROC: 0.7654410209924843, SD AUROC: 0.013638873769267172, Mean AUPRC: 0.026246303892671834, SD AUPRC: 0.007520281770493733


## Part III: PE

In [6]:
results=K_fold_val("PE", df) 

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   5.2s
[CV] END .learning_rate=0.1, max_depth=4

1it [25:42, 1542.93s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   6.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   6.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   5.1s
[CV] END .learning_rate=0.1, max_depth=4

2it [51:28, 1544.60s/it]

...

test


Step,Training Loss
1000,10.252500
2000,6.307400
3000,5.064800
4000,4.487900
5000,4.089100
6000,3.800300
7000,3.554300
8000,3.381500
9000,3.222700
10000,3.121500


100%|██████████| 531/531 [01:40<00:00,  5.30it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   5.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   5.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   4.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   5.1s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [9:32:30, 6870.00s/it]


In [7]:
print(results) 

metrics: Mean AUROC: 0.7283001598328548, SD AUROC: 0.03238080502197492, Mean AUPRC: 0.01271810914944842, SD AUPRC: 0.0018802334515895236


## Part 4: PNA

In [5]:
results=K_fold_val("PNA", df) 

0it [00:00, ?it/s]You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing CustomClinicalBertForCombinedLearning: ['distilbert.transformer.layer.3.sa_layer_norm.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.5.ffn.lin1.weight', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.2.ffn.lin1.bias', 'distilbert.transformer.layer.3.attention.k_lin.bias', 'distilbert.transformer.layer.3.ffn.lin2.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.4.output_layer_norm.weight', 'distilbert.transformer.layer.5.attention.k_lin.weight', 'distilbert.transformer.layer.2.attention.k_

test


Step,Training Loss
1000,9.783400
2000,5.884100
3000,4.947400
4000,4.424300
5000,4.105500
6000,3.813100
7000,3.602000
8000,3.477500
9000,3.347800
10000,3.279700


100%|██████████| 531/531 [01:02<00:00,  8.45it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.6s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

test


Step,Training Loss
1000,9.746500
2000,5.890800
3000,4.944000
4000,4.419800
5000,4.070300
6000,3.815100
7000,3.586500
8000,3.463200
9000,3.340000
10000,3.254100


100%|██████████| 531/531 [00:53<00:00,  9.94it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [9:46:09, 7033.81s/it]


In [6]:
print(results) 
# lambda=5 
    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     num_train_epochs=7,
    #     per_device_train_batch_size=40,
    #     per_device_eval_batch_size=32,
    #     warmup_steps=1000,
    #     weight_decay=0.001,
    #     learning_rate=0.00001,
    #     logging_dir="./logs",
    #     logging_steps=1000,
    #     save_strategy="epoch")


metrics: Mean AUROC: 0.8137837176087995, SD AUROC: 0.008332573963755936, Mean AUPRC: 0.04280324968217599, SD AUPRC: 0.00977034304162096


## Part 5: Post_AKI_status

In [9]:
results=K_fold_val("post_aki_status", df) 

0it [00:00, ?it/s]You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing CustomClinicalBertForCombinedLearning: ['distilbert.transformer.layer.2.ffn.lin1.weight', 'distilbert.transformer.layer.3.ffn.lin1.bias', 'distilbert.transformer.layer.5.attention.q_lin.weight', 'distilbert.transformer.layer.2.attention.v_lin.bias', 'distilbert.transformer.layer.4.attention.q_lin.weight', 'vocab_projector.bias', 'distilbert.transformer.layer.4.ffn.lin1.weight', 'distilbert.transformer.layer.3.attention.v_lin.weight', 'distilbert.transformer.layer.0.output_layer_norm.weight', 'distilbert.transformer.layer.4.ffn.lin2.bias', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.2.output_layer_norm.weight', 'distilbert.transformer.layer.2.sa_layer_norm.weight', 'distilbert.tran

test


Step,Training Loss
1000,7.473100
2000,3.799000
3000,2.000500
4000,1.406800
5000,1.162500
6000,1.002300
7000,0.909400
8000,0.833800
9000,0.763300
10000,0.728900


100%|██████████| 531/531 [00:53<00:00,  9.88it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

test


Step,Training Loss
1000,7.491100
2000,3.756000
3000,2.002400
4000,1.417200
5000,1.174700
6000,1.025700
7000,0.893300
8000,0.824500
9000,0.768400
10000,0.715200


100%|██████████| 531/531 [00:53<00:00,  9.86it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [7:43:36, 5563.27s/it]


In [10]:
print(results) 

    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     num_train_epochs=7,
    #     per_device_train_batch_size=32,
    #     per_device_eval_batch_size=32,
    #     warmup_steps=1500,
    #     weight_decay=0.1,
    #     learning_rate=0.0001,
    #     logging_dir="./logs",
    #     logging_steps=1000,
    #     save_strategy="epoch")

    # LAMBDA=1

metrics: Mean AUROC: 0.8379338860154777, SD AUROC: 0.004311230774676896, Mean AUPRC: 0.4889776539597821, SD AUPRC: 0.012682365974097132


## Part 6: delirium

In [11]:
df = df.dropna(subset=['postop_del']).reset_index(drop=True)
df["postop_del"]=df["postop_del"].astype(int)
results=K_fold_val("postop_del", df)

0it [00:00, ?it/s]You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing CustomClinicalBertForCombinedLearning: ['distilbert.transformer.layer.2.ffn.lin1.weight', 'distilbert.transformer.layer.3.ffn.lin1.bias', 'distilbert.transformer.layer.5.attention.q_lin.weight', 'distilbert.transformer.layer.2.attention.v_lin.bias', 'distilbert.transformer.layer.4.attention.q_lin.weight', 'vocab_projector.bias', 'distilbert.transformer.layer.4.ffn.lin1.weight', 'distilbert.transformer.layer.3.attention.v_lin.weight', 'distilbert.transformer.layer.0.output_layer_norm.weight', 'distilbert.transformer.layer.4.ffn.lin2.bias', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.2.output_layer_norm.weight', 'distilbert.transformer.layer.2.sa_layer_norm.weight', 'distilbert.tran

test


Step,Training Loss
1000,7.474900


100%|██████████| 77/77 [00:07<00:00, 10.07it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

test


Step,Training Loss
1000,7.500600


100%|██████████| 77/77 [00:07<00:00, 10.06it/s]


Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [1:17:18, 927.60s/it]


In [12]:
print(results) 

    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     num_train_epochs=7,
    #     per_device_train_batch_size=32,
    #     per_device_eval_batch_size=32,
    #     warmup_steps=1500,
    #     weight_decay=0.1,
    #     learning_rate=0.0001,
    #     logging_dir="./logs",
    #     logging_steps=1000,
    #     save_strategy="epoch")

    # LAMBDA=1

metrics: Mean AUROC: 0.6930331252708833, SD AUROC: 0.005665955299380421, Mean AUPRC: 0.6708903830059538, SD AUPRC: 0.008228124926259567
